In [4]:
import pandas as pd
import numpy as np
import requests as rq
import xml.etree.ElementTree as ET

## Funções

In [5]:
def parse_XML(url, df_cols, path_inicio): 
    
    resp = rq.get(url)
    conteudoXML = resp.content
    tree = ET.ElementTree(ET.fromstring(conteudoXML))
    xroot = tree.getroot()
    inicio = xroot.findall(path_inicio)
        
    #xtree = et.parse(xml_file)
    #xroot = xtree.getroot()
    rows = []
    
    for node in inicio: 
        res = []
        
        for el in df_cols: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [6]:
def get_detalhes_materia_completo(codigo_materia):
    url = "https://legis.senado.leg.br/dadosabertos/materia/%d" % codigo_materia
    col_rename = {
        'IdentificacaoMateria/DescricaoIdentificacaoMateria': 'Identificacao',
        'DadosBasicosMateria/EmentaMateria': 'Ementa',
        'DadosBasicosMateria/IndexacaoMateria': 'Indexacao',
        'DadosBasicosMateria/NaturezaMateria/DescricaoNatureza': 'Natureza',
        'Classificacoes/Classificacao/DescricaoClasse': 'Classe',
        'Classificacoes/Classificacao/DescricaoClasseHierarquica': 'ClasseHierarquica',
        'DadosBasicosMateria/Autor': 'Autor',
        'DadosBasicosMateria/ExplicacaoEmentaMateria': 'ExplicacaoEmenta'
    }
    df = parse_XML(url, list(col_rename.keys()), "Materia")
    df.rename(columns=col_rename, inplace=True)
    df['link'] = "https://www25.senado.leg.br/web/atividade/materias/-/materia/%d" % codigo_materia
    df['CodigoMateria'] = codigo_materia    
    return df

## Área de testes

In [4]:
url = "https://legis.senado.leg.br/dadosabertos/materia/situacaoatual/153151"
df_cols = ["Tramitando"]
path_inicio = "Materias/Materia"
resp = rq.get(url)
conteudoXML = resp.content
tree = ET.ElementTree(ET.fromstring(conteudoXML))
xroot = tree.getroot()
inicio = xroot.findall(path_inicio)


rows = []

for node in inicio: 
    res = []

    for el in df_cols: 
        
        if node is not None and node.find(el) is not None:
            res.append(node.find(el).text)
        else: 
            res.append(None)
    rows.append({df_cols[i]: res[i] 
                 for i, _ in enumerate(df_cols)})

out_df = pd.DataFrame(rows, columns=df_cols)

In [5]:
out_df

,Tramitando
0,Nao


In [7]:
df = get_detalhes_materia_completo(136033)
df

,Identificacao,Ementa,Indexacao,Natureza,Classe,ClasseHierarquica,Autor,ExplicacaoEmenta,link,CodigoMateria
0,PL 1864/2019,"Altera o Decreto-Lei nº 2.848, de 7 de dezembr...","ALTERAÇÃO , CODIGO PENAL , CODIGO DE PROCES...",Norma Geral,Direito Penal e Penitenciário,Jurídico / Direito Penal e Penitenciário,"Senadora Eliziane Gama (CIDADANIA/MA), Senador...","Estabelece medidas contra a corrupção, o crime...",https://www25.senado.leg.br/web/atividade/mate...,136033
